# **Install MMDetection and Import Relevant Libraries**

In [ ]:
!nvcc -V
!gcc --version

In [ ]:
!pip install -U torch==1.7.1+cu110 torchvision==0.8.2+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install mmcv-full
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection
%cd mmdetection
!pip install -e .
!pip install Pillow==7.0.0

In [ ]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

import mmdet as mmdet
print(mmdet.__version__)

from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

import os
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
import glob
import cv2
import shutil
import random
import os.path as osp
import json
import mmcv
import random
import re
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
from typing import Dict, List
from mmdet.apis import inference_detector, init_detector, show_result_pyplot, set_random_seed

In [ ]:
%cd ..

In [ ]:
global_seed = 0

def set_seed(seed=global_seed):
    """Sets the random seeds."""
    set_random_seed(seed, deterministic=False)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed()

# **Preprocess Data**

In [ ]:
!mkdir /kaggle/working/data
!mkdir /kaggle/working/data/train-xml-labels
!cp -a ../input/cgi-planes-in-satellite-imagery-w-bboxes/train/. /kaggle/working/data/train-xml-labels
!mkdir /kaggle/working/data/val-xml-labels
!cp -a ../input/cgi-planes-in-satellite-imagery-w-bboxes/test/. /kaggle/working/data/val-xml-labels
!mkdir /kaggle/working/data/train-images
!mkdir /kaggle/working/data/val-images

In [ ]:
%%writefile labels.txt
plane

In [ ]:
for file in os.listdir('/kaggle/working/data/train-xml-labels'):
    if file[-3:] == 'png':
        shutil.move('/kaggle/working/data/train-xml-labels/' + file, '/kaggle/working/data/train-images')
        
for file in os.listdir('/kaggle/working/data/val-xml-labels'):
    if file[-3:] == 'png':
        shutil.move('/kaggle/working/data/val-xml-labels/' + file, '/kaggle/working/data/val-images')

In [ ]:
f = open("train.txt", "x")
f.close()
lines = []
for file in os.listdir('/kaggle/working/data/train-xml-labels'):
    lines.append('/kaggle/working/data/train-xml-labels/' + file)
with open('train.txt', 'w') as f:
    for line in lines:
        f.write(line)
        f.write('\n')
f.close()

In [ ]:
f = open("val.txt", "x")
f.close()
lines = []
for file in os.listdir('/kaggle/working/data/val-xml-labels'):
    lines.append('/kaggle/working/data/val-xml-labels/' + file)
with open('val.txt', 'w') as f:
    for line in lines:
        f.write(line)
        f.write('\n')
f.close()

# **Convert from PASCAL VOC to COCO**

In [ ]:
#Stolen from https://github.com/yukkyo/voc2coco
def get_label2id(labels_path: str) -> Dict[str, int]:
    with open(labels_path, 'r') as f:
        labels_str = f.read().split()
    labels_ids = list(range(1, len(labels_str)+1))
    return dict(zip(labels_str, labels_ids))


def get_annpaths(ann_dir_path: str = None,
                 ann_ids_path: str = None,
                 ext: str = '',
                 annpaths_list_path: str = None) -> List[str]:
    # If use annotation paths list
    if annpaths_list_path is not None:
        with open(annpaths_list_path, 'r') as f:
            ann_paths = f.read().split()
        return ann_paths

    # If use annotaion ids list
    ext_with_dot = '.' + ext if ext != '' else ''
    with open(ann_ids_path, 'r') as f:
        ann_ids = f.read().split()
    ann_paths = [os.path.join(ann_dir_path, aid+ext_with_dot) for aid in ann_ids]
    return ann_paths


def get_image_info(annotation_root, extract_num_from_imgid=True, train_or_val='train'):
    filename = annotation_root.findtext('filename')
    if train_or_val == 'train':
        filename = '/kaggle/working/data/train-images/' + filename
    else:
        filename = '/kaggle/working/data/val-images/' + filename 
    img_name = os.path.basename(filename)
    img_id = os.path.splitext(img_name)[0]
    if extract_num_from_imgid and isinstance(img_id, str):
        img_id = int(re.findall(r'\d+', img_id)[0])

#     size = annotation_root.find('size')
#     width = int(size.findtext('width'))
#     height = int(size.findtext('height'))
    width, height, depth = cv2.imread(filename).shape

    image_info = {
        'id': img_id,
        'width': width,
        'height': height,
        'file_name': filename,
    }
    return image_info


def get_coco_annotation_from_obj(obj, label2id):
    label = obj.findtext('name')
#     assert label in label2id, f"Error: {label} is not in label2id !"
    category_id = label2id[label]
    bndbox = obj.find('bndbox')
    xmin = int(float(bndbox.findtext('xmin')))
    ymin = int(float(bndbox.findtext('ymin')))
    xmax = int(float(bndbox.findtext('xmax')))
    ymax = int(float(bndbox.findtext('ymax')))
    assert xmax >= xmin and ymax >= ymin, f"Box size error !: (xmin, ymin, xmax, ymax): {xmin, ymin, xmax, ymax}"
    o_width = xmax - xmin
    o_height = ymax - ymin
    ann = {
        'category_id': category_id,
        'segmentation': [],  # This script is not for segmentation
        'area': o_width * o_height,
        'bbox': [xmin, ymin, o_width, o_height],
        'iscrowd': 0,
    }
    return ann


def convert_xmls_to_cocojson(annotation_paths: List[str],label2id: Dict[str, int], output_jsonpath: str, train_or_val: str, extract_num_from_imgid: bool = True):
    output_json_dict = { 
        "images": [],
        "annotations": [],
        "categories": []
    }
    bnd_id = 1  # START_BOUNDING_BOX_ID, TODO input as args ?
    print('Start converting !')
    for a_path in annotation_paths:
        # Read annotation xml
        ann_tree = ET.parse(a_path)
        ann_root = ann_tree.getroot()

        img_info = get_image_info(annotation_root=ann_root,extract_num_from_imgid=extract_num_from_imgid, train_or_val=train_or_val)
        img_id = img_info['id']
        output_json_dict['images'].append(img_info)

        for obj in ann_root.findall('object'):
            ann = get_coco_annotation_from_obj(obj=obj, label2id=label2id)
            annot = {'id': bnd_id, 'image_id': img_id,}
            annot.update(ann)
            output_json_dict['annotations'].append(annot)
            bnd_id = bnd_id + 1

    for label, label_id in label2id.items():
        category_info = {'id': label_id, 'name': label, 'supercategory': 'none'}
        output_json_dict['categories'].append(category_info)

    with open(output_jsonpath, 'w') as f:
        output_json = json.dumps(output_json_dict)
        f.write(output_json)

In [ ]:
def convert_to_coco(ann_path_list='/kaggle/working/train.txt', labels='/kaggle/working/labels.txt', output='/kaggle/working/output.json', train_or_val='train'):
    label2id = get_label2id(labels_path=labels)
    ann_paths = get_annpaths(
        annpaths_list_path=ann_path_list
    )
    convert_xmls_to_cocojson(
        annotation_paths=ann_paths,
        label2id=label2id,
        output_jsonpath=output,
        train_or_val=train_or_val
    )

In [ ]:
convert_to_coco()

In [ ]:
convert_to_coco(ann_path_list='/kaggle/working/val.txt', labels='/kaggle/working/labels.txt', output='/kaggle/working/val_output.json', train_or_val='val')

# **Create Custom Config File**

In [ ]:
from mmcv import Config
cfg = Config.fromfile('/kaggle/working/mmdetection/configs/dcn/cascade_rcnn_r101_fpn_dconv_c3-c5_1x_coco.py')

In [ ]:
from mmdet.apis import set_random_seed

cfg.dataset_type = 'CocoDataset'
cfg.classes = '/kaggle/working/labels.txt'
cfg.data_root = '/kaggle/working'

for head in cfg.model.roi_head.bbox_head:
    head.num_classes = 1

cfg.data.test.type = 'CocoDataset'
cfg.data.test.classes = 'labels.txt'
cfg.data.test.data_root = '/kaggle/working'
cfg.data.test.ann_file = 'val_output.json'
cfg.data.test.img_prefix = 'images'

cfg.data.train.type = 'CocoDataset'
cfg.data.train.data_root = '/kaggle/working'
cfg.data.train.ann_file = 'output.json'
cfg.data.train.img_prefix = 'images'
cfg.data.train.classes = 'labels.txt'

cfg.data.val.type = 'CocoDataset'
cfg.data.val.data_root = '/kaggle/working'
cfg.data.val.ann_file = 'val_output.json'
cfg.data.val.img_prefix = 'images'
cfg.data.val.classes = 'labels.txt'

albu_train_transforms = [
    dict(type='ShiftScaleRotate', shift_limit=0.0625,
         scale_limit=0.15, rotate_limit=15, p=0.4),
    dict(type='RandomBrightnessContrast', brightness_limit=0.2,
         contrast_limit=0.2, p=0.5),
    dict(type='IAAAffine', shear=(-10.0, 10.0), p=0.4),
#     dict(type='MixUp', p=0.2, lambd=0.5),
    dict(type="Blur", p=1.0, blur_limit=7),
    dict(type='CLAHE', p=0.5),
    dict(type='Equalize', mode='cv', p=0.4),
    dict(
        type="OneOf",
        transforms=[
            dict(type="GaussianBlur", p=1.0, blur_limit=7),
            dict(type="MedianBlur", p=1.0, blur_limit=7),
        ],
        p=0.4,
    ),]

cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=True),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Albu',
        transforms=albu_train_transforms,
        bbox_params=dict(
        type='BboxParams',
        format='coco',
        label_fields=['gt_labels'],
        min_visibility=0.0,
        filter_lost_elements=True),
        keymap=dict(img='image', gt_bboxes='bboxes'),
        update_pad_shape=False,
        skip_img_without_anno=True),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]

cfg.load_from = '../input/dcn-checkpoint/cascade_rcnn_r101_fpn_dconv_c3-c5_1x_coco_20200203-3b2f0594.pth'

cfg.work_dir = '/kaggle/working/model_output'

cfg.optimizer.lr = 0.02 / 8
cfg.lr_config = dict(
    policy='CosineAnnealing', 
    by_epoch=False,
    warmup='linear', 
    warmup_iters=500, 
    warmup_ratio=0.001,
    min_lr=1e-07)

cfg.data.samples_per_gpu = 4
cfg.data.workers_per_gpu = 2

cfg.evaluation.metric = 'bbox'
cfg.evaluation.interval = 4

cfg.checkpoint_config.interval = 6
cfg.runner.max_epochs = 18
cfg.log_config.interval = 50

cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = [0]

print(f'Config:\n{cfg.pretty_text}')

# **Train Model**

In [ ]:
datasets = [build_dataset(cfg.data.train)]
model = build_detector(
 cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
model.CLASSES = datasets[0].CLASSES

mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

# **Inference on Test Set**

In [ ]:
model = init_detector(cfg, '/kaggle/working/model_output/epoch_18.pth')
val_set = os.listdir('/kaggle/working/data/val-images')
inference_set = random.sample(val_set, 30)
for file in inference_set:
    img = mmcv.imread('/kaggle/working/data/val-images/' + file)
    result = inference_detector(model, img)
    show_result_pyplot(model, img, result)